In [56]:
import h5py
import numpy as np
import torch
import seqexplainer as se

from eugene import models

In [33]:
model = models.DeepSTARR.load_from_checkpoint("/cellar/users/aklie/projects/ML4GLand/use_cases/deAlmeida22/models/eugene/DeepSTARR.ckpt")

In [37]:
dataset = h5py.File("/cellar/users/aklie/projects/ML4GLand/use_cases/deAlmeida22/data/evo_aug/DeepSTARR_data.h5", 'r')
x_test = np.array(dataset['X_test']).astype(np.float32)
y_test = np.array(dataset['Y_test']).astype(np.float32)
dataset.close()

In [60]:
test_np = x_test[:100]
test_torch = torch.from_numpy(test_np)

In [64]:
import importlib
from seqexplainer import _references as ref
importlib.reload(ref)

<module 'seqexplainer._references' from '/cellar/users/aklie/projects/ML4GLand/SeqExplainer/seqexplainer/_references.py'>

In [70]:
ref.zero_ref_inputs(test_np).shape, zero_ref_inputs(test_torch).shape

(torch.Size([100, 4, 249]), torch.Size([100, 4, 249]))

In [68]:
ref.random_ref_inputs(test_np).shape, ref.random_ref_inputs(test_torch).shape

torch.Size([100, 4, 249])

In [71]:
ref.shuffle_ref_inputs(test_np).shape, ref.shuffle_ref_inputs(test_torch).shape

((100, 4, 249), torch.Size([100, 4, 249]))

In [73]:
ref.dinuc_shuffle_ref_inputs(test_np).shape

torch.Size([100, 4, 249])

In [76]:
ref.gc_ref_inputs(test_np, bg="uniform", uniform_dist=[0.3, 0.2, 0.2, 0.3]).shape, ref.gc_ref_inputs(test_torch, bg="uniform", uniform_dist=[0.3, 0.2, 0.2, 0.3]).shape

(torch.Size([100, 4, 249]), torch.Size([100, 4, 249]))

In [79]:
ref.gc_ref_inputs(test_np, bg="batch").shape, ref.gc_ref_inputs(test_torch, bg="batch").shape

(torch.Size([100, 4, 249]), torch.Size([100, 4, 249]))

In [86]:
# Numpy one currently broken
ref.gc_ref_inputs(test_torch, bg="seq").shape, ref.gc_ref_inputs(test_np, bg="seq").shape

TypeError: _sum() got an unexpected keyword argument 'dim'

In [ ]:
se.attribute(model, x_test[0:1], method="NaiveISM", reference_type="shuffle", target=0, batch_size=128, device="cuda")